# Capstone Project - The Battle of the Neighborhoods
## Applied Data Science Capstone
### Location recommendation for new CrossFit box in Toronto C.A. by Suzie Amey

## Table of contents
* [Introduction: Business Problem](#introduction)
* [Data](#data)
* [Methodology](#methodology)
* [Results](#results)
* [Discussion](#discussion)
* [Conclusion](#conclusion)



## Introduction: Business Problem <a name="introduction"></a>

### Problem <a name="problem"></a>

A client is looking to open a new **CrossFit** training gym in the city of **Toronto, Ontario**, and has asked for us to come up with an ideal location based on statistics and data from the area.  A CrossFit training gym is known as a Box, and in order to call themselves a CrossFit Box, each gym must pay a yearly membership fee to the CrossFit brand which makes them an affiliate.  As such, **Crossfit memberships for individuals are known to be quite pricey**, hence, the ideal location would likely be in an area where the **household income** is on the higher end of the scale.  Also, it would be good to find a location which is **not too close to any other CrossFit Box**, as the residents of that area may already be members somewhere else.  *It would also be a good idea to consider whether the neighbourhood is one **where health and fitness is a priority**, so one where there are a number of sports and fitness shops and health food stores would be a good fit*.

### Audience <a name="audience"></a>

The audience in this case is the client who is looking to open a new Box.  Their obvious reason for interest is to open the Box in an area where they will bring in enough clients to make their gym a success.

## Data <a name="data"></a>

### Data Sources <a name="data sources"></a>

As mentioned, the **average income** of a neighbourhood is going to be a factor in deciding where to open a new CrossFit Box; the following website has information on just such information:

  * [Statistics Canada](https://www03.cmhc-schl.gc.ca/hmip-pimh/en/TableMapChart/TableMatchingCriteria?GeographyType=MetropolitanMajorArea&GeographyId=2270&CategoryLevel1=Population%2C%20Households%20and%20Housing%20Stock&CategoryLevel2=Household%20Income&ColumnField=HouseholdIncomeRange&RowField=Neighbourhood&SearchTags%5B0%5D.Key=Households&SearchTags%5B0%5D.Value=Number&SearchTags%5B1%5D.Key=Statistics&SearchTags%5B1%5D.Value=AverageAndMedian)
  
Another column in this table which may be beneficial to consider is **Population Density**, and this information can be obtained from the following website:

  * [Wikipedia Demographics of Toronto Neighbourhoods](https://en.wikipedia.org/wiki/Demographics_of_Toronto_neighbourhoods)

Opening a Box in an area with low population density means there would be fewer potential clients to attract.
I will also use Foursquare location data to find out which neighbourhoods already have a CrossFit Box, and also, I will look for areas which show an interest in personal health and fitness; for example, these areas could have health food and sports equipment stores and a number of other outlets for physical activities.

### Data Cleaning and Analysis <a name="data cleaning and analysis"></a>

I will read the income information into a Pandas Data frame in my Jupyter notebook and keep only the columns which are relevant to the given problem.  I will create a map using this data to give a visual representation of neighbourhoods which may be suitable.  
From the preliminary neighbourhoods given by the initial data cleaning, I will then use Foursquare to search for venues of interest within these locations.

## Methodology <a name="methodology"></a>

Firstly, lets import any libraries which may be needed througout the notebook

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    certifi-2019.11.28         |           py36_0         149 KB  conda-forge
    ca-certificates-2019.11.28 |       hecc5488_0         145 KB  conda-forge
    geopy-1.21.0               |             py_0          58 KB  conda-forge
    openssl-1.1.1d             |       h516909a_0         2.1 MB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.50-py_0         conda-forge
    geopy:           1.21.0-py_0       conda-forge

The following packages will be UPDATED:

    ca-

Next, I'll read in the data from the **Statistics Canada** website and clean it up so I'm left with the information I need, which is *'**Median Household Income Before Taxes**'*.  I'll also rename the columns in the dataframe to make it easier to read.

In [75]:
url = "https://www03.cmhc-schl.gc.ca/hmip-pimh/en/TableMapChart/TableMatchingCriteria?GeographyType=MetropolitanMajorArea&GeographyId=2270&CategoryLevel1=Population%2C%20Households%20and%20Housing%20Stock&CategoryLevel2=Household%20Income&ColumnField=HouseholdIncomeRange&RowField=Neighbourhood&SearchTags%5B0%5D.Key=Households&SearchTags%5B0%5D.Value=Number&SearchTags%5B1%5D.Key=Statistics&SearchTags%5B1%5D.Value=AverageAndMedian"
df = pd.read_html(url)[0]

df.head()

,Unnamed: 0,Average Household Income Before Taxes,Median Household Income Before Taxes,Average Household Income After Taxes,Median Household Income After Taxes
0,Agincourt/Malvern,80147,68108,71025,62304
1,Ajax/Pickering,115659,98363,94805,84664
2,Alderwood,102434,83249,84163,73039
3,Aurora,147604,106761,113594,90592
4,Banbury-Don Mills/York Mills,245866,90620,168106,76046


In [76]:
df.drop(['Average Household Income Before Taxes', 'Average Household Income After Taxes', 'Median Household Income After Taxes'], axis=1, inplace = True)
df.head(5)

,Unnamed: 0,Median Household Income Before Taxes
0,Agincourt/Malvern,68108
1,Ajax/Pickering,98363
2,Alderwood,83249
3,Aurora,106761
4,Banbury-Don Mills/York Mills,90620


In [77]:
df.rename(columns = {"Unnamed: 0" : "Neighbourhood", "Median Household Income Before Taxes" : "Income"}, inplace = True) 
df.head(5)

,Neighbourhood,Income
0,Agincourt/Malvern,68108
1,Ajax/Pickering,98363
2,Alderwood,83249
3,Aurora,106761
4,Banbury-Don Mills/York Mills,90620


The next step was to read in a list of postal codes for each neighbourhood in Canada.  This will be used to get the **coordinates** of each neighbourhood when creating a map.  I also did a bit of tidying up of this dataframe to get rid of any values which were **'Not assigned'**.

In [80]:
url1 = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
df1 = pd.read_html(url1)[0]

df1.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [81]:
df_seg = df1[df1.Borough != 'Not assigned']
df_seg.head(10)

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor
7,M7A,Downtown Toronto,Queen's Park
9,M9A,Queen's Park,Not assigned
10,M1B,Scarborough,Rouge
11,M1B,Scarborough,Malvern
13,M3B,North York,Don Mills North


In [82]:
df_seg['Neighbourhood'].replace('Not assigned',df_seg['Borough'],inplace=True)
df_seg.head(5)

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor


In [83]:
df_seg['Neighbourhood'].groupby([df_seg.Postcode,df_seg.Borough]).apply(list).reset_index()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"[Rouge, Malvern]"
1,M1C,Scarborough,"[Highland Creek, Rouge Hill, Port Union]"
2,M1E,Scarborough,"[Guildwood, Morningside, West Hill]"
3,M1G,Scarborough,[Woburn]
4,M1H,Scarborough,[Cedarbrae]
5,M1J,Scarborough,[Scarborough Village]
6,M1K,Scarborough,"[East Birchmount Park, Ionview, Kennedy Park]"
7,M1L,Scarborough,"[Clairlea, Golden Mile, Oakridge]"
8,M1M,Scarborough,"[Cliffcrest, Cliffside, Scarborough Village West]"
9,M1N,Scarborough,"[Birch Cliff, Cliffside West]"


In [84]:
df2 = df_seg.rename(columns = {"Postcode": "Postal Code"}) 

I then read in a table of data with **Latitude and Longitude** values for each **Postal Code**, and then joined all the data together so that I had, for each Neighbourhood, a Latitude and Longitude value, as well as the average income.

In [85]:
df1 = pd.read_csv('https://cocl.us/Geospatial_data')
df1.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [86]:
neighbourhoods = pd.merge(df2, df1, on='Postal Code')
neighbourhoods.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
3,M6A,North York,Lawrence Heights,43.718518,-79.464763
4,M6A,North York,Lawrence Manor,43.718518,-79.464763


In [87]:
nhoods = pd.merge(neighbourhoods, df, on='Neighbourhood')
nhoods.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude,Income
0,M4A,North York,Victoria Village,43.725882,-79.315572,51867
1,M1B,Scarborough,Rouge,43.806686,-79.194353,101482
2,M3C,North York,Flemingdon Park,43.725900,-79.340923,48917
3,M6C,York,Humewood-Cedarvale,43.693781,-79.428191,61110
4,M1E,Scarborough,Guildwood,43.763573,-79.188711,87538


In [88]:
nhoods.drop(['Postal Code', 'Borough'], axis=1, inplace = True)
nhoods.head()

,Neighbourhood,Latitude,Longitude,Income
0,Victoria Village,43.725882,-79.315572,51867
1,Rouge,43.806686,-79.194353,101482
2,Flemingdon Park,43.725900,-79.340923,48917
3,Humewood-Cedarvale,43.693781,-79.428191,61110
4,Guildwood,43.763573,-79.188711,87538


Once I had all the information I needed for each Neighbourhood, I could sort in order from **Highest Income to Lowest**, and pick the top 5 most affluent Neighbourhoods.  These were the ones I would then investigate further for suitability.

In [89]:
nhoods.sort_values(by=['Income'], ascending = False, inplace=True)
nhoods.head(10)

,Neighbourhood,Latitude,Longitude,Income
27,Rosedale,43.679563,-79.377529,114625
1,Rouge,43.806686,-79.194353,101482
7,The Beaches,43.676357,-79.293031,100365
25,Moore Park,43.689574,-79.383160,97762
4,Guildwood,43.763573,-79.188711,87538
28,Alderwood,43.602414,-79.543484,83249
16,Riverdale,43.679557,-79.352188,77819
19,Cliffcrest,43.716316,-79.239476,75291
23,Forest Hill North,43.696948,-79.411307,70920
9,Bathurst Manor,43.754328,-79.442259,69634


In [90]:
nhoods = nhoods.reset_index(drop=True)

In [91]:
top_five = nhoods[:5]
top_five

,Neighbourhood,Latitude,Longitude,Income
0,Rosedale,43.679563,-79.377529,114625
1,Rouge,43.806686,-79.194353,101482
2,The Beaches,43.676357,-79.293031,100365
3,Moore Park,43.689574,-79.383160,97762
4,Guildwood,43.763573,-79.188711,87538


Next, I created a map of Toronto, using Folium, with markers for the locations of the top 5 Neighbourhoods

In [92]:
address = 'Toronto, CA'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [93]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, neighbourhood in zip(top_five['Latitude'], top_five['Longitude'], top_five['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, 'Toronto')
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

Then, it was time to use Foursquare to find out whether any of the 5 Neighbourhoods already had a **CrossFit Box**.

In [94]:
# The code was removed by Watson Studio for sharing.

In [95]:
address1 = 'Rosedale, Toronto'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address1)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Rosedale are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Rosedale are 43.6783556, -79.3807457.


In [96]:
search_query = 'CrossFit'
radius = 500

In [97]:
url2 = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius)
url2

'https://api.foursquare.com/v2/venues/search?client_id=QELSOPIU4YTD3H0NS03HBFGWEHUEREWU51ACI10SV2BCGFIA&client_secret=RP02DMBFW3WDUSAAYHOSVZSKEYFHUHWEK1JJWZ5FPS0RQQB4&ll=43.6783556,-79.3807457&v=20180605&query=CrossFit&radius=500'

In [98]:
results = requests.get(url2).json()
results

{'meta': {'code': 200, 'requestId': '5e5129b7216785001c4c7cec'},
 'response': {'venues': [{'id': '4b7f01d9f964a520e00f30e3',
    'name': 'uptown fitness',
    'location': {'lat': 43.672672,
     'lng': -79.379627,
     'labeledLatLngs': [{'label': 'display',
       'lat': 43.672672,
       'lng': -79.379627}],
     'distance': 639,
     'cc': 'CA',
     'country': 'Canada',
     'formattedAddress': ['Canada']},
    'categories': [],
    'referralId': 'v-1582377412',
    'hasPerk': False},
   {'id': '4c5854ecb13695218ed4735a',
    'name': 'Booty Camp Fitness',
    'location': {'address': 'David Balfour Park',
     'lat': 43.68341,
     'lng': -79.3829,
     'labeledLatLngs': [{'label': 'display',
       'lat': 43.68341,
       'lng': -79.3829}],
     'distance': 588,
     'cc': 'CA',
     'city': 'Toronto',
     'state': 'ON',
     'country': 'Canada',
     'formattedAddress': ['David Balfour Park', 'Toronto ON', 'Canada']},
    'categories': [{'id': '4bf58dd8d48988d175941735',
      'n

In [99]:
address2 = 'Rouge, Toronto'

geolocator = Nominatim(user_agent="foursquare_agent")
location1 = geolocator.geocode(address2)
latitude1 = location1.latitude
longitude1 = location1.longitude
print('The geograpical coordinate of Rouge are {}, {}.'.format(latitude1, longitude1))

The geograpical coordinate of Rouge are 43.8049304, -79.1658374.


In [100]:
url3 = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}'.format(CLIENT_ID, CLIENT_SECRET, latitude1, longitude1, VERSION, search_query, radius)
url3

'https://api.foursquare.com/v2/venues/search?client_id=QELSOPIU4YTD3H0NS03HBFGWEHUEREWU51ACI10SV2BCGFIA&client_secret=RP02DMBFW3WDUSAAYHOSVZSKEYFHUHWEK1JJWZ5FPS0RQQB4&ll=43.8049304,-79.1658374&v=20180605&query=CrossFit&radius=500'

In [101]:
results1 = requests.get(url3).json()
results1

{'meta': {'code': 200, 'requestId': '5e5129fbb57e880021413377'},
 'response': {'venues': []}}

In [102]:
address3 = 'The Beaches, Toronto'

geolocator = Nominatim(user_agent="foursquare_agent")
location2 = geolocator.geocode(address3)
latitude2 = location2.latitude
longitude2 = location2.longitude
print('The geograpical coordinate of The Beaches are {}, {}.'.format(latitude2, longitude2))

The geograpical coordinate of The Beaches are 43.6710244, -79.296712.


In [103]:
url4 = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}'.format(CLIENT_ID, CLIENT_SECRET, latitude2, longitude2, VERSION, search_query, radius)
url4

'https://api.foursquare.com/v2/venues/search?client_id=QELSOPIU4YTD3H0NS03HBFGWEHUEREWU51ACI10SV2BCGFIA&client_secret=RP02DMBFW3WDUSAAYHOSVZSKEYFHUHWEK1JJWZ5FPS0RQQB4&ll=43.6710244,-79.296712&v=20180605&query=CrossFit&radius=500'

In [104]:
results2 = requests.get(url4).json()
results2

{'meta': {'code': 200, 'requestId': '5e5128f4963d29001bef6a01'},
 'response': {'venues': [{'id': '516ddce8e4b03e302bf5d68c',
    'name': 'Prana Fitness',
    'location': {'lat': 43.67107,
     'lng': -79.295043,
     'labeledLatLngs': [{'label': 'display',
       'lat': 43.67107,
       'lng': -79.295043}],
     'distance': 134,
     'cc': 'CA',
     'country': 'Canada',
     'formattedAddress': ['Canada']},
    'categories': [{'id': '4bf58dd8d48988d102941735',
      'name': 'Yoga Studio',
      'pluralName': 'Yoga Studios',
      'shortName': 'Yoga Studio',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/gym_yogastudio_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': 'v-1582377423',
    'hasPerk': False},
   {'id': '4c0d0c6a98102d7f388be106',
    'name': 'Kew Beach Off Leash Dog Park',
    'location': {'lat': 43.6658149758116,
     'lng': -79.29698824882507,
     'labeledLatLngs': [{'label': 'display',
       'lat': 43.6658149758116,
     

In [105]:
address4 = 'Moore Park, Toronto'

geolocator = Nominatim(user_agent="foursquare_agent")
location3 = geolocator.geocode(address4)
latitude3 = location3.latitude
longitude3 = location3.longitude
print('The geograpical coordinate of Moore Park are {}, {}.'.format(latitude3, longitude3))

The geograpical coordinate of Moore Park are 43.6903876, -79.3832965.


In [106]:
url5 = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}'.format(CLIENT_ID, CLIENT_SECRET, latitude3, longitude3, VERSION, search_query, radius)
url5

'https://api.foursquare.com/v2/venues/search?client_id=QELSOPIU4YTD3H0NS03HBFGWEHUEREWU51ACI10SV2BCGFIA&client_secret=RP02DMBFW3WDUSAAYHOSVZSKEYFHUHWEK1JJWZ5FPS0RQQB4&ll=43.6903876,-79.3832965&v=20180605&query=CrossFit&radius=500'

In [107]:
results3 = requests.get(url5).json()
results3

{'meta': {'code': 200, 'requestId': '5e5129c3c94979001bbebc1f'},
 'response': {'venues': [{'id': '4bb0200cf964a52090383ce3',
    'name': 'Pure Fitness',
    'location': {'address': '939 Eglinton Ave E',
     'crossStreet': 'Brentcliffe',
     'lat': 43.693006599930136,
     'lng': -79.38778127126965,
     'labeledLatLngs': [{'label': 'display',
       'lat': 43.693006599930136,
       'lng': -79.38778127126965}],
     'distance': 464,
     'postalCode': 'M4G 4E8',
     'cc': 'CA',
     'city': 'Toronto',
     'state': 'ON',
     'country': 'Canada',
     'formattedAddress': ['939 Eglinton Ave E (Brentcliffe)',
      'Toronto ON M4G 4E8',
      'Canada']},
    'categories': [{'id': '4bf58dd8d48988d176941735',
      'name': 'Gym',
      'pluralName': 'Gyms',
      'shortName': 'Gym',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/building/gym_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': 'v-1582377427',
    'hasPerk': False}]}}

In [108]:
address5 = 'Guildwood, Toronto'

geolocator = Nominatim(user_agent="foursquare_agent")
location4 = geolocator.geocode(address5)
latitude4 = location4.latitude
longitude4 = location4.longitude
print('The geograpical coordinate of Guildwood are {}, {}.'.format(latitude4, longitude4))

The geograpical coordinate of Guildwood are 43.7552251, -79.1982293.


In [109]:
url6 = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}'.format(CLIENT_ID, CLIENT_SECRET, latitude4, longitude4, VERSION, search_query, radius)
url6

'https://api.foursquare.com/v2/venues/search?client_id=QELSOPIU4YTD3H0NS03HBFGWEHUEREWU51ACI10SV2BCGFIA&client_secret=RP02DMBFW3WDUSAAYHOSVZSKEYFHUHWEK1JJWZ5FPS0RQQB4&ll=43.7552251,-79.1982293&v=20180605&query=CrossFit&radius=500'

In [110]:
results4 = requests.get(url6).json()
results4

{'meta': {'code': 200, 'requestId': '5e5129a2ad1ab4001bbb2a9f'},
 'response': {'venues': []}}

From running the queries above, we can see that, out of the 5 Neighbourhoods, only Rouge (*address2*), and Guildwood (*address5*) don't already have a **CrossFit Box**, so these would be our two shortlisted locations to recommend.  I then tried running a query to see whether there were any other venues in these two areas which showed an interest in **Sports**, with the following results:

In [111]:
search_query1 = 'sport'
radius = 500

In [112]:
url7 = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}'.format(CLIENT_ID, CLIENT_SECRET, latitude1, longitude1, VERSION, search_query1, radius)
url7

'https://api.foursquare.com/v2/venues/search?client_id=QELSOPIU4YTD3H0NS03HBFGWEHUEREWU51ACI10SV2BCGFIA&client_secret=RP02DMBFW3WDUSAAYHOSVZSKEYFHUHWEK1JJWZ5FPS0RQQB4&ll=43.8049304,-79.1658374&v=20180605&query=sport&radius=500'

In [113]:
results5 = requests.get(url7).json()
results5

{'meta': {'code': 200, 'requestId': '5e512a2d1a4b0a001b25510a'},
 'response': {'venues': []}}

In [114]:
url8 = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}'.format(CLIENT_ID, CLIENT_SECRET, latitude4, longitude4, VERSION, search_query1, radius)
url8

'https://api.foursquare.com/v2/venues/search?client_id=QELSOPIU4YTD3H0NS03HBFGWEHUEREWU51ACI10SV2BCGFIA&client_secret=RP02DMBFW3WDUSAAYHOSVZSKEYFHUHWEK1JJWZ5FPS0RQQB4&ll=43.7552251,-79.1982293&v=20180605&query=sport&radius=500'

In [115]:
results6 = requests.get(url8).json()
results6

{'meta': {'code': 200, 'requestId': '5e5129b17828ae001b629440'},
 'response': {'venues': [{'id': '4b721eecf964a520b16f2de3',
    'name': "Sunny's Sport Bar and Grill",
    'location': {'address': '4190 Kingston Rd.',
     'lat': 43.75917706072689,
     'lng': -79.19765072624969,
     'labeledLatLngs': [{'label': 'display',
       'lat': 43.75917706072689,
       'lng': -79.19765072624969}],
     'distance': 442,
     'postalCode': 'M1E 4W1',
     'cc': 'CA',
     'city': 'Toronto',
     'state': 'ON',
     'country': 'Canada',
     'formattedAddress': ['4190 Kingston Rd.',
      'Toronto ON M1E 4W1',
      'Canada']},
    'categories': [{'id': '4bf58dd8d48988d11d941735',
      'name': 'Sports Bar',
      'pluralName': 'Sports Bars',
      'shortName': 'Sports Bar',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/nightlife/sportsbar_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': 'v-1582377436',
    'hasPerk': False}]}}

Only **Guildwood** had any hits on sports-related venues.  So, this is the location which I am going to recommend to my client as a suitable location to open a new **CrossFit Box**.